In [ ]:
import numpy as np
import pandas as pd
import re
import sklearn
import spacy
import nltk

In [ ]:
#Import training data 
#allData = pd.read_csv("perfect_done.txt", delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

In [ ]:
# Text preprocessing and adds speaker column
speakers = []
for i in range(len(allData)):
    if (':' in allData.iloc[i, 0]):
        index = allData.iloc[i, 0].index(':')
        speakers.append((allData.iloc[i, 0])[:index+1])
        allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0]) 
    else:
        speakers.append('F:')
    allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0])
allData.insert (0, "speaker", speakers)

In [ ]:
class Sentence:
# Creates a Sentence object
# text = text of Sentence
# rp_bin = which occurence of done is being looked at in the sentence
  # eg. if the variable be is set to 2, the model will observe whether the second noun/verb clause contains an instance of perfect done
# num = the index of the Sentence in the original csv
    remoteBin = 0
    r1 = 0; r2 = 0;
    def __init__(self, text, d, num, true):
        self.text = text
        self.num = num
        self.d = d
        self.true = true

In [ ]:
# Preproccesses training text for linguistic analysis

lines = []

with open("perfect_done.txt", "r", encoding="utf-8") as tsv_file: #these files are tab separated but you can easily change this to CSV
    tsv_reader = csv.reader(tsv_file, delimiter="\t")
    header = next(tsv_reader)
    for row in tsv_reader:
        row = row[:1]
        row[0] = re.sub("[a-zA-Z]:\s+", "", row[0]) #removes the interviewer tag
        row[0] = re.sub("\s{2,}", " ", row[0]) #removes excessive spaces
        row[0] = re.sub("’|‘", "'", row[0]) #fixes apostrophes
        row[0] = re.sub("—", "--", row[0]) #m-dash was causing formatting issues, changed it to two dashes
        row[0] = re.sub("“", '"', row[0]) #fixes quotation marks    
        lines.append(row[0])
         
print(len(lines))

In [ ]:
# Create an array of Sentence objects with at least one 'done'

sen = []
for line in range(len(lines)):
        d = 1
        numDone = 0
        parsed = nlp(lines[line])
        for i, word in enumerate(parsed):
            if (word.text.lower() == "done"):
                numDone += 1
        if (numDone >= 1):
            sen.append(Sentence(lines[line], d, line))
            while (numDone > 1):
                d += 1
                sen.append(Sentence(lines[line], d, line))
                numDone -= 1
for Sentence in range(len(sen)):
    sen[Sentence].text = (sen[Sentence].text).replace('"','')

In [ ]:
# Tags the Sentence objects with each rule

index = -1
senNum = 1
numDone = 0
doneIndices = []

for Sentence in sen:
    index += 1 
    numDone = 0
    if (index > 0 and (sen[index].text == sen[index-1].text) and (sen[index].num == sen[index-1].num)):
        senNum += 1
    else:
        senNum = 1
    parsed = nlp(Sentence.text)
    for i, word in enumerate(parsed):
        if (i >= 0 and (word.text.lower() in ('done')) and Sentence.perfectDone != 1):
            numBin += 1
            if (numDone == senNum):
                w.append(word.pos_)
                doneIndices.append(i)
                doneChildren = list(word.children)
                if(word.head == word):
                    doneSibling = []
                else:
                    doneSibling = list(word.head.children)
                doneDep = word.dep_
                donePOS = word.pos_
                doneIndex = i
                num = 1
                next = None
                prev = None
                if (prev.pos_ == 'NOUN' or prev.pos_ == 'PRON' or prev.pos_ == 'PROPN') and (next.pos_ == 'VERB' or next.pos_ == 'AUX')
                    r1 = 1
#                if (Sentence.r2 != 1):
#                    Sentence.r2 = 0
                if (Sentence.r1 == 1):# and Sentence.r2 != 1):
                    Sentence.perfectDone = 1
                else:
                    Sentence.perfectDone = -1

In [ ]:
# Consolidation done values of duplicated sentences
# e.g. if any have disagreement, assign all done values to 1, else keep them as 0

multDoneinSentence = []

for i in range(len(sen)-1):
    if (i < len(sen)-1):
        j = i+1 
        if (sen[i].text != sen[j].text):
            if (sen[i].d == 1):
                multDoneinSentence.append(True)
            else:
                multDoneinSentence.append(False)
        while (sen[i].text == sen[j].text):
            if (sen[i].d == 1 or sen[j].d == 1):
                multDoneinSentence.append(True)
            else:
                multDoneinSentence.append(False)
            if (sen[j].d == 1):
                sen[i].d == sen[j].d
            if (sen[i].d == 1):
                sen[j].d == sen[i].d
            if (j < len(sen)-1):
                j +=1
            else:
                break

if len(multDoneinSentence) != len(sen):
    if (sen[-1].d == 1):
        multDoneinSentence.append(True)
    else:
        multDoneinSentence.append(False)

In [ ]:
# Goes back to the original data and annotates each sentences with perfect done or not

perfectDone = []
for i in range(len(allData)):
    m = 0
    for j in range(len(sen)):
        if i == sen[j].num:
            if (m != 1):
                m = sen[j].d
            if (m == 1 and ("%PeD" not in allData.iloc[i, allData.columns.get_loc('Speaker')])):
                allData.loc[i, 'Speaker'] = allData.iloc[i, allData.columns.get_loc('Speaker')] + " %PeD"
    perfectDone.append(m)
allData['perfectDone'] = perfectDone

In [ ]:
# Print classification report

predictions = allData['perfectDone']
y = allData['perfectDONE']
target_names = ['perfectDone', 'none']
print(classification_report(y, predictions, target_names=target_names))

In [ ]:
with open('perfectDone.txt', 'w+', newline='') as file:
   allData.to_csv('perfectDone.txt', sep = '\t')